In [1]:
#Import python libraries
import dvc.api
import pandas as pd
from pandas_profiling import ProfileReport
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

In [4]:
# import requiered module
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from load_data import LoadData
from preprocessing import Preprocess
from plot_data import PlotData

In [6]:
loader = LoadData()
preprocess = Preprocess()
plot = PlotData()

In [7]:
train_path = 'data/train_merged.csv'
test_path = 'data/test_merged.csv'
repo = 'https://github.com/yonamg/Pharmaceutical-Sales-Prediction'
version = 'tr_mv1'